# PROJET DE STATISTIQUE DES RISQUES EXTREMES

Réalisé par : *Mariane ALAPINI & Céleste NENEHIDINI*

Sous la supervision de  : **Nicolas JEANNELLE**

## Introduction

Notre objectif sur ce projet est de tester plusieurs méthodes d'estimation de la Value-at-Risk (VaR) pour un portefeuille donné. La VaR est une mesure synthétique du risque de marché. Elle est définie comme la perte potentielle maximale que peut subir un portefeuille pour un niveau de confiance donné sur un horizon fixé.

Mathématiquement, soit $F$ la fonction de répartition de la variable aléatoire $R$ des rendements de périodicité $h$ du portefeuille. La VaR de niveau de confiance $α$ à horizon d’investissement $h$ est alors définie par :

$$
\mathrm{VaR}_h(\alpha) = F^{-1}(1 - \alpha)
$$

On adopte dans ce cas une vision **rentabilité** où $R$ est la distribution des rendements. On peut aussi adopter une vision **risque** en considérant $P = −R$, la distribution des pertes et
dans ce cas :

$$
\mathrm{VaR}_h(\alpha) = G^{-1}(\alpha)
$$

avec $G$ la fonction de répartition de $P$.

Dans la suite, sauf mention contraire, nous raisonnerons d'un point de vue **risque**.

## 0. Analyses préliminaires

**Importation des bibliothèques**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import binomtest
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.special import gamma
from scipy.stats import gumbel_r

**a. Importation des données**

Pour ce projet, nous avons choisi d'étudier le cours du **S&P 500**, dans sa version ETF. Le S&P 500 (Standard & Poor’s 500) est l’un des indices boursiers les plus suivis au monde. Il regroupe 500 des plus grandes entreprises cotées aux États-Unis, sélectionnées selon des critères de capitalisation boursière, de liquidité et de représentativité sectorielle. L’indice est pondéré par la capitalisation boursière, ce qui signifie que les entreprises les plus importantes (comme Apple, Microsoft ou Amazon) ont un poids plus élevé dans son évolution. De ce fait, le S&P 500 est généralement considéré comme un baromètre de la performance globale du marché actions américain, et plus largement de l’économie américaine.

Plutôt que d’analyser directement la valeur théorique de l’indice, nous nous intéressons ici à sa version ETF (Exchange Traded Fund) car elle est directement observable et négociable, avec des données de prix, de volumes et de rendements disponibles. Nous avons considéré les données disponibles sur Yahoo Finance pour les dix dernières années.

In [ ]:
TICKER = "SPY"

df = yf.download(TICKER, period="10y", interval="1d", auto_adjust=False)

if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

last_close = df["Close"].iloc[-1]
print(f"Dernier valeur à la clôture du {TICKER}: {last_close:.2f}")


df.to_csv(f"price_{TICKER}.csv")
print("CSV créé:", f"price_{TICKER}.csv")

**b. Vérification de la présence de valeurs manquantes**

In [ ]:
df["Close"].isna().sum()

Nous n'avons pas de valeurs manquantes dans la série des prix à la clotûre. Il n'y a donc aucun traitement particulier à effectuer avant de débuter les analyses.

**c. Calcul du log-rendement**

In [ ]:
df["log_return"] = np.log(df["Close"] / df["Close"].shift(1))
# Supprimer la première ligne qui est NaN après le calcul des log-rendements
df = df.dropna(subset=["log_return"])
df["log_return" ].head()

**d. Représentation graphique**

In [ ]:
# Create figure
fig, ax1 = plt.subplots(figsize=(13, 6))

# Prix à le cloture 
ax1.plot(
    df.index,
    df["Close"],
    color="blue",
    linewidth=2,
    label="Price"
)
ax1.set_xlabel("Date")
ax1.set_ylabel("Price (USD)", color="blue")
ax1.tick_params(axis="y", labelcolor="blue")
ax1.grid(True)

# Log-rendements
ax2 = ax1.twinx()
ax2.plot(
    df.index,
    df["log_return"],
    color="red",
    alpha=0.6,
    label="Log return"
)
ax2.axhline(0, color="red", linestyle="--", linewidth=1)
ax2.set_ylabel("Log return", color="red")
ax2.tick_params(axis="y", labelcolor="red")

# Legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

plt.title("SPY – Prix (en bleu) et log-rendements (rouge)")
plt.tight_layout()
plt.show()

**e. Méthode d'identification automatique des périodes de forte volatilité**

In [ ]:
def detect_volatility_clusters(
    df,
    return_col="log_return",
    short_window=60,
    long_window=200,
    ratio_threshold=1.5,
    min_duration=3
):
    """
    Detect volatility clusters based on local vs long-term volatility divergence.

    Returns:
    - df with volatility measures and cluster flag
    - list of (start_date, end_date) clusters
    """
    df = df.copy()

    # Short-term and long-term volatility
    df["vol_short"] = df[return_col].rolling(short_window, center=False).std()
    df["vol_long"] = df[return_col].rolling(long_window, center=True).std()

    # Volatility ratio
    df["vol_ratio"] = df["vol_short"] / df["vol_long"]

    # Initial high-vol flag
    df["high_vol_flag"] = df["vol_ratio"] > ratio_threshold

    # ---- Extract clusters (bags) ----
    clusters = []
    start = None
    duration = 0

    for date, is_high in df["high_vol_flag"].items():
        if is_high:
            if start is None:
                start = date
                duration = 1
            else:
                duration += 1
        else:
            if start is not None and duration >= min_duration:
                clusters.append((start, date))
            start = None
            duration = 0

    # Edge case
    if start is not None and duration >= min_duration:
        clusters.append((start, df.index[-1]))

    return df, clusters

Explication de la démarche : à insérer

In [ ]:
# Application de la détection des clusters
df_clusters, clusters = detect_volatility_clusters(
    df,
    return_col="log_return",
    short_window=60,
    long_window=200,
    ratio_threshold=1.2,
    min_duration=60
)

In [ ]:
clusters

In [ ]:
def plot_returns_with_clusters(df, clusters, return_col="log_return", title=None):
    """
    Plot log-returns and highlight detected volatility clusters.
    - df: DataFrame indexé par dates
    - clusters: liste de tuples (start_date, end_date)
    """
    fig, ax = plt.subplots(figsize=(14, 5))

    # Série de rendements
    ax.plot(df.index, df[return_col], linewidth=1, label=return_col)

    # Surlignage des clusters (bandes verticales)
    for (start, end) in clusters:
        ax.axvspan(start, end, alpha=0.25)  # pas de couleur imposée

    ax.axhline(0, linewidth=1)
    ax.set_xlabel("Date")
    ax.set_ylabel("Log-return")
    ax.set_title(title if title else "Log-returns avec périodes de forte volatilité")
    ax.legend()
    plt.show()


In [ ]:
plot_returns_with_clusters(df_clusters, clusters, return_col="log_return",
                           title="Train: log-returns + clusters de volatilité")

**f. Découpage train/test**

On prend une période de forte volatilité d'au moins 60 jours, précédée d'une période de faible volatilité de 250 jours. 

In [ ]:
df_simple = df[["Close", "log_return"]].copy()
df_simple.head()

In [ ]:
split_date1 = "2020-06-11"
split_date2 = "2025-04-08"
split_date3 = "2025-07-08"

train = df_simple.loc[
    (df_simple.index >= split_date1) &
    (df_simple.index <= split_date2)
]

test = df_simple.loc[
    (df_simple.index > split_date2) &
    (df_simple.index <= split_date3)
]

In [ ]:
train.shape, test.shape

On a bien deux échantillons de train et de test qui respectent les conditions requises.

**g. Statistiques descriptives sur les deux échantillons**

In [ ]:
boxplot_data = [train["log_return"], test["log_return"]]
labels = ['Train Set', 'Test Set']
plt.figure(figsize=(8, 6))
plt.boxplot(boxplot_data, labels=labels)
plt.title('Boxplot of Log Returns: Train vs Test Set')
plt.ylabel('Log Return')
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
sns.histplot(train["log_return"], bins=50, color='blue', label='Train Set', kde=True, stat="density", alpha=0.6)
sns.histplot(test["log_return"], bins=50, color='orange', label='Test Set', kde=True, stat="density", alpha=0.6)
plt.title('Histogram of Log Returns: Train vs Test Set')
plt.legend()
plt.show()

La distribution au niveau du test est beaucoup plus étalée et possède des queues plus lourdes.

In [ ]:
train.describe()

In [ ]:
test.describe()

## 1. Calcul de la VAR non paramétrique

**a. Fonction calculant la VaR historique d’un ensemble de log-rendements**

In [ ]:
def VaR_Hist(log_returns, alpha=0.99):
    """
    VaR historique à 1 jour au niveau alpha à partir de log-rendements.
    
    Paramètres
    ----------
    log_returns : array-like ou pd.Series
        Série de log-rendements (r_t).
    alpha : float
        Niveau de confiance (ex: 0.99 pour VaR 99%).

    Retour
    ------
    float
        VaR (positive), i.e. perte telle que P(L <= VaR) = alpha avec L = -r.
    """
    r = pd.Series(log_returns).dropna()

    # Quantile de la queue gauche des rendements
    q = np.quantile(r, 1 - alpha)

    # VaR en pertes (positive)
    return float(-q)


**b. VaR historique sur base d’apprentissage pour alpha = 99%**

In [ ]:
VaR_99 = VaR_Hist(train["log_return"], alpha=0.99)
print(VaR_99)

**c. Fonction calculant la VaR historique bootstrap d’un ensemble de log-rendements et donnant un IC de niveau alpha_IC de cette VaR**

In [ ]:
def VaR_Hist_Bootstrap(log_returns, alpha=0.99, alpha_IC=0.95, B=2000, random_state=None):
    """
    VaR historique à 1 jour au niveau alpha + intervalle de confiance bootstrap (percentile)
    de niveau alpha_IC.

    Paramètres
    ----------
    log_returns : array-like ou pd.Series
        Série de log-rendements r_t.
    alpha : float
        Niveau de confiance de la VaR (ex: 0.99).
    alpha_IC : float
        Niveau de confiance de l'intervalle (ex: 0.95).
    B : int
        Nombre de réplications bootstrap.
    random_state : int ou None
        Graine pour reproductibilité.

    Retour
    ------
    var_hat : float
        VaR historique (positive), avec L = -r.
    ic : tuple(float, float)
        (borne_inf, borne_sup) de l'IC bootstrap percentile.
    """
    r = pd.Series(log_returns).dropna().to_numpy()
    n = r.size
    if n < 2:
        raise ValueError("La série doit contenir au moins 2 rendements non-NaN.")

    rng = np.random.default_rng(random_state)

    # Estimateur VaR historique sur l'échantillon
    q = np.quantile(r, 1 - alpha)
    var_hat = float(-q)

    # Bootstrap
    vars_boot = np.empty(B, dtype=float)
    for b in range(B):
        sample = rng.choice(r, size=n, replace=True)
        vars_boot[b] = -np.quantile(sample, 1 - alpha)

    # IC percentile (central)
    tail = (1 - alpha_IC) / 2
    lower = float(np.quantile(vars_boot, tail))
    upper = float(np.quantile(vars_boot, 1 - tail))

    return var_hat, (lower, upper)


In [ ]:
var_99, ic_95 = VaR_Hist_Bootstrap(train["log_return"], alpha=0.99, alpha_IC=0.95, B=500, random_state=42)
print("VaR 99% :", var_99)
print("IC 95% :", ic_95)

**d. VaR historique bootstrap et l’IC associé à 90% sur base d’apprentissage pour alpha = 99%**

In [ ]:
var_99, ic_90 = VaR_Hist_Bootstrap(train["log_return"], alpha=0.99, alpha_IC=0.90, B=500, random_state=42)
print("VaR 99% :", var_99)
print("IC 90% :", ic_90)

**e. Nombre d’exceptions sur base de test associées à la VaR historique calculée**

In [ ]:
def nb_exceptions(test_returns, VaR):
    """
    Compte le nombre d'exceptions associées à une VaR donnée.

    Paramètres
    ----------
    test_returns : array-like ou pd.Series
        Log-rendements de l'échantillon test.
    VaR : float
        VaR à 1 jour (positive).

    Retour
    ------
    n_exc : int
        Nombre d'exceptions.
    rate_exc : float
        Taux d'exceptions.
    """
    r_test = pd.Series(test_returns).dropna().to_numpy()

    exceptions = r_test < -VaR
    n_exc = int(exceptions.sum())
    rate_exc = n_exc / len(r_test) if len(r_test) > 0 else np.nan

    return n_exc, rate_exc

In [ ]:
n_exc, rate = nb_exceptions(test["log_return"], VaR_99)
print("Nombre d'exceptions :", n_exc)
print("Taux d'exceptions   :", rate)

**f. Comparaison avec le niveau de risque attendu (IC de binomiale)**

In [ ]:
k = n_exc
n = test.shape[0]
p0 = 0.01          # attendu pour VaR 99%
alpha_IC = 0.9

# IC exact Clopper-Pearson
res = binomtest(k, n, p=p0, alternative="greater")
ci = res.proportion_ci(confidence_level=alpha_IC, method="exact")

print("Taux observé:", k/n)
print(f"IC {alpha_IC:.0%} (Clopper-Pearson): [{ci.low:.4f}, {ci.high:.4f}]")
print("p-value :", res.pvalue)


Le taux attendu sous une VaR 99% est 1%, et 1% n’appartient pas à cet intervalle de confiance calculée. Statistiquement, le taux d’exceptions observé est incompatible avec une VaR 99% bien calibrée. On a beaucoup trop d’exceptions et donc la VaR sous-estime le risque sur le test.

## 2. Calcul de la VaR Gaussienne

**a. Fonction calculant la VaR gaussienne d’un ensemble de log-rendements**

In [ ]:
def VaR_Gauss(log_returns, alpha=0.99):
    """
    VaR gaussienne à 1 jour au niveau alpha.

    Paramètres
    ----------
    log_returns : array-like ou pd.Series
        Série de log-rendements.
    alpha : float
        Niveau de confiance (ex: 0.99).

    Retour
    ------
    float
        VaR gaussienne (positive).
    """
    r = pd.Series(log_returns).dropna()

    mu = r.mean()
    sigma = r.std(ddof=1)

    z = norm.ppf(1 - alpha)

    VaR = -(mu + sigma * z)
    return float(VaR)

**b. VaR gaussienne sur base d’apprentissage pour alpha = 99%**

In [ ]:
VaR_gauss_99 = VaR_Gauss(train["log_return"], alpha=0.99)
print(VaR_gauss_99)

**c. Validation ex-ante (analyses graphiques, QQ-plot, etc)**

In [ ]:
def validation_ex_ante_gaussienne(log_returns, alpha=0.99, window_vol=20):
    """
    Validation ex-ante pour l'hypothèse gaussienne sur une série de log-rendements.
    - Série de rendements
    - Volatilité glissante (clustering)
    - Histogramme + fit normal
    - QQ-plot vs Normal(0,1) (standardisé)
    - Marque le quantile (1-alpha) théorique sous normalité (queue gauche)
    """
    r = pd.Series(log_returns).dropna()
    n = len(r)

    mu = r.mean()
    sigma = r.std(ddof=1)

    # standardisation
    z = (r - mu) / sigma

    # quantile queue gauche (rendements)
    q_emp = np.quantile(r, 1 - alpha)
    q_norm = mu + sigma * stats.norm.ppf(1 - alpha)

    # ---- 1) Rendements (time series) ----
    plt.figure(figsize=(12, 4))
    plt.plot(r.index, r.values)
    plt.axhline(q_emp, linestyle="--")
    plt.axhline(q_norm, linestyle="--")
    plt.title(f"Log-rendements (n={n}) et quantile (1-α) | α={alpha}")
    plt.xlabel("Date")
    plt.ylabel("log_return")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # ---- 2) Volatilité glissante ----
    vol_roll = r.rolling(window_vol).std()
    plt.figure(figsize=(12, 4))
    plt.plot(vol_roll.index, vol_roll.values)
    plt.title(f"Volatilité glissante (std sur {window_vol} jours) — détection clustering")
    plt.xlabel("Date")
    plt.ylabel("rolling std")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # ---- 3) Histogramme + fit normal ----
    plt.figure(figsize=(8, 4.8))
    plt.hist(r.values, bins=60, density=True)

    xs = np.linspace(r.min(), r.max(), 400)
    plt.plot(xs, stats.norm.pdf(xs, loc=mu, scale=sigma))
    plt.axvline(q_emp, linestyle="--", linewidth=1)
    plt.axvline(q_norm, linestyle="--", linewidth=1)
    plt.title("Histogramme des rendements + densité normale ajustée")
    plt.xlabel("log_return")
    plt.ylabel("densité")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # ---- 4) QQ-plot (standardisé) ----
    plt.figure(figsize=(6, 6))
    stats.probplot(z.values, dist="norm", plot=plt)
    plt.title("QQ-plot des rendements standardisés vs N(0,1)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # ---- 5) Statistiques simples (queue / asymétrie) ----
    skew = stats.skew(r.values, bias=False)
    kurt = stats.kurtosis(r.values, fisher=True, bias=False)  # excess kurtosis

    print(f"mu = {mu:.6f}, sigma = {sigma:.6f}")
    print(f"Quantile empirique (1-α) : {q_emp:.6f}")
    print(f"Quantile normal ajusté (1-α) : {q_norm:.6f}")
    print(f"Asymétrie (skewness) : {skew:.3f}")
    print(f"Kurtosis excédentaire : {kurt:.3f}  (0 si normal)")


In [ ]:
validation_ex_ante_gaussienne(train["log_return"], alpha=0.99, window_vol=20)

**d. VaR gaussienne à 10j par la méthode du scaling**

In [ ]:
VaR_gauss_10j = VaR_gauss_99 * np.sqrt(10)
print(VaR_gauss_10j)

**e. VaR gaussienne à 10j par méthode de diffusion d’un actif**

In [ ]:
def var_gaussienne_diffusion(S0, mu, sigma, alpha=0.99, n_sims=200000, seed=42):
    rng = np.random.default_rng(seed)
    T = 10  # jours
    Z = rng.standard_normal(n_sims)

    # GBM exact sur horizon T
    ST = S0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    # perte (positive si baisse)
    loss = S0 - ST

    # VaR = quantile alpha des pertes
    return float(np.quantile(loss, alpha))

In [ ]:
S0 = train["Close"].iloc[-1]
mu = train["log_return"].mean()
sigma = train["log_return"].std(ddof=1)

VaR_10j_99 = var_gaussienne_diffusion(S0, mu, sigma, alpha=0.99, n_sims=20000, seed=42)
print(VaR_10j_99)

Calcul précédent à revoir !!

In [ ]:
def var_gaussienne_10j_rendement(mu, sigma, alpha=0.99):
    T = 10
    z = norm.ppf(1 - alpha)  # négatif
    return float(-(mu*T + sigma*np.sqrt(T)*z))  # VaR en % (positive)

VaR_10j_99 = var_gaussienne_10j_rendement(mu, sigma, alpha=0.99)
print(VaR_10j_99)

In [ ]:
S0 = float(train["Close"].iloc[-1])
mu = float(train["log_return"].mean())
sigma = float(train["log_return"].std(ddof=1))

VaR_10j_99 = var_gaussienne_diffusion(S0, mu, sigma, alpha=0.99, n_sims=20000, seed=42)
print(VaR_10j_99)


## 3. VaR skew-Student

**a. Estimation des paramètres d’une loi de Skew Student par maximum de vraisemblance**

In [ ]:
# ----------------------------
# Skewed Student-t de Hansen (1994)
# Paramètres:
#   mu    : localisation
#   sigma : échelle (>0)
#   nu    : ddl (>2)
#   lam   : skewness dans (-1, 1)
# ----------------------------

def _hansen_constants(nu, lam):
    """
    Calcule a, b, c de Hansen (1994) pour la skewed t.
    """
    c = gamma((nu + 1) / 2) / (np.sqrt(np.pi * (nu - 2)) * gamma(nu / 2))
    a = 4 * lam * c * (nu - 2) / (nu - 1)
    b = np.sqrt(1 + 3 * lam**2 - a**2)
    return a, b, c

def skewt_pdf(x, mu, sigma, nu, lam):
    """
    PDF de la skewed t de Hansen (1994) évaluée en x (vectorisé).
    """
    x = np.asarray(x)
    if sigma <= 0 or nu <= 2 or not (-1 < lam < 1):
        return np.zeros_like(x, dtype=float)

    a, b, c = _hansen_constants(nu, lam)
    z = (x - mu) / sigma

    # seuil de bascule
    thresh = -a / b

    left = z < thresh
    right = ~left

    pdf = np.empty_like(z, dtype=float)

    # partie gauche
    denom_L = (1 - lam)
    arg_L = (b * z[left] + a) / denom_L
    pdf[left] = (b * c / sigma) * (1 + (arg_L**2) / (nu - 2)) ** (-(nu + 1) / 2)

    # partie droite
    denom_R = (1 + lam)
    arg_R = (b * z[right] + a) / denom_R
    pdf[right] = (b * c / sigma) * (1 + (arg_R**2) / (nu - 2)) ** (-(nu + 1) / 2)

    return pdf

def skewt_loglik(params, x):
    """
    Log-vraisemblance (somme des log pdf).
    params = (mu, log_sigma, log_nu_minus2, atanh_lam)
    """
    mu, log_sigma, log_nu_m2, atanh_lam = params
    sigma = np.exp(log_sigma)
    nu = 2.0 + np.exp(log_nu_m2)      # garantit nu > 2
    lam = np.tanh(atanh_lam)          # garantit lam in (-1, 1)

    pdf = skewt_pdf(x, mu, sigma, nu, lam)
    # sécurité numérique
    eps = 1e-14
    return np.sum(np.log(pdf + eps))

def fit_skewt_mle(x, mu0=None, sigma0=None, nu0=8.0, lam0=0.0):
    """
    Estime (mu, sigma, nu, lam) par MLE avec scipy.optimize.minimize.
    """
    x = np.asarray(x)

    if mu0 is None:
        mu0 = np.mean(x)
    if sigma0 is None:
        sigma0 = np.std(x, ddof=1)

    # paramètres transformés initiaux
    p0 = np.array([
        mu0,
        np.log(max(sigma0, 1e-8)),
        np.log(max(nu0 - 2.0, 1e-6)),
        np.arctanh(np.clip(lam0, -0.999, 0.999))
    ], dtype=float)

    # on maximise loglik => on minimise -loglik
    def objective(p):
        return -skewt_loglik(p, x)

    res = minimize(objective, p0, method="L-BFGS-B")

    mu_hat, log_sigma_hat, log_nu_m2_hat, atanh_lam_hat = res.x
    sigma_hat = np.exp(log_sigma_hat)
    nu_hat = 2.0 + np.exp(log_nu_m2_hat)
    lam_hat = np.tanh(atanh_lam_hat)

    out = {
        "mu": mu_hat,
        "sigma": sigma_hat,
        "nu": nu_hat,
        "lam": lam_hat,
        "success": res.success,
        "message": res.message,
        "fun": res.fun,  # valeur min de -loglik
        "loglik": -res.fun
    }
    return out

**b. Paramètres de loi Skew Student sur base d’apprentissage**

In [ ]:
# Fit skew-t
skewt_fit = fit_skewt_mle(train['log_return'], nu0=8.0, lam0=0.0)
print("Skew-t MLE:", skewt_fit)

# Fit gaussien (MLE fermé)
mu_g, sigma_g = np.mean(train['log_return']), np.std(train['log_return'], ddof=1)
print("Gaussian:", {"mu": mu_g, "sigma": sigma_g})

**c. Validation ex-ante par QQ-plot**

In [ ]:
def sample_skewt_hansen(n, mu, sigma, nu, lam, rng=None):
    """
    Génération par transformation:
    - On génère une Student-t symétrique t_nu
    - On applique la transformation piecewise de Hansen sur z
    - puis x = mu + sigma*z
    """
    if rng is None:
        rng = np.random.default_rng()

    a, b, c = _hansen_constants(nu, lam)

    # Student t symétrique standard (moyenne 0)
    # On génère via ratio normal / sqrt(chi2/nu)
    z0 = rng.standard_normal(n)
    u  = rng.chisquare(df=nu, size=n)
    t  = z0 / np.sqrt(u / nu)

    # On applique la transformation inverse de la définition piecewise
    # Hansen définit la densité en fonction de z, avec seuil -a/b
    # Ici on transforme t -> z en inversant l'argument piecewise.
    # On utilise la même séparation via probabilité implicite:
    # plus simple: on utilise la construction équivalente:
    #   z = ((1-lam)*t - a)/b si t < 0
    #   z = ((1+lam)*t - a)/b si t >= 0
    # Cette construction est couramment utilisée avec cette paramétrisation.
    z = np.empty_like(t)
    neg = t < 0
    z[neg]  = ((1 - lam) * t[neg]  - a) / b
    z[~neg] = ((1 + lam) * t[~neg] - a) / b

    return mu + sigma * z

def qqplot_gaussian(x, mu, sigma, title="QQ-plot Gaussian"):
    x = np.asarray(x)
    x_sorted = np.sort(x)
    n = len(x_sorted)
    p = (np.arange(1, n+1) - 0.5) / n
    theo = norm.ppf(p, loc=mu, scale=sigma)

    plt.figure()
    plt.plot(theo, x_sorted, marker='o', linestyle='none')
    # droite 45° (référence)
    mn = min(theo.min(), x_sorted.min())
    mx = max(theo.max(), x_sorted.max())
    plt.plot([mn, mx], [mn, mx])
    plt.title(title)
    plt.xlabel("Quantiles théoriques")
    plt.ylabel("Quantiles empiriques")
    plt.show()

def qqplot_skewt_sim(x, fit, nsim=200000, seed=123, title="QQ-plot Skew-t (simulation)"):
    x = np.asarray(x)
    x_sorted = np.sort(x)
    n = len(x_sorted)
    p = (np.arange(1, n+1) - 0.5) / n

    rng = np.random.default_rng(seed)
    sim = sample_skewt_hansen(nsim, fit["mu"], fit["sigma"], fit["nu"], fit["lam"], rng=rng)
    sim_sorted = np.sort(sim)

    # quantiles simulés aux mêmes probabilités
    idx = np.clip((p * (nsim - 1)).astype(int), 0, nsim - 1)
    theo = sim_sorted[idx]

    plt.figure()
    plt.plot(theo, x_sorted, marker='o', linestyle='none')
    mn = min(theo.min(), x_sorted.min())
    mx = max(theo.max(), x_sorted.max())
    plt.plot([mn, mx], [mn, mx])
    plt.title(title)
    plt.xlabel("Quantiles théoriques (skew-t simulée)")
    plt.ylabel("Quantiles empiriques")
    plt.show()

In [ ]:
# QQ-plots ex-ante sur test
qqplot_gaussian(test['log_return'], mu_g, sigma_g, title="QQ-plot ex-ante (Gaussian) - test")
qqplot_skewt_sim(test['log_return'], skewt_fit, nsim=300000, seed=1, title="QQ-plot ex-ante (Skew-t Hansen) - test")

In [ ]:
# QQ-plots ex-ante sur train
qqplot_gaussian(train['log_return'], mu_g, sigma_g, title="QQ-plot ex-ante (Gaussian) - train")
qqplot_skewt_sim(train['log_return'], skewt_fit, nsim=300000, seed=1, title="QQ-plot ex-ante (Skew-t Hansen) - train")

**d. Comparaison de la qualité de fit entre loi gaussienne et loi de skew Student par analyse graphique**

In [ ]:
def plot_fit_comparison(x, mu_g, sigma_g, skewt_fit, bins=60, title="Fit comparison (train)"):
    x = np.asarray(x)
    grid = np.linspace(np.percentile(x, 0.5), np.percentile(x, 99.5), 800)

    pdf_g = norm.pdf(grid, loc=mu_g, scale=sigma_g)
    pdf_s = skewt_pdf(grid, skewt_fit["mu"], skewt_fit["sigma"], skewt_fit["nu"], skewt_fit["lam"])

    plt.figure()
    plt.hist(x, bins=bins, density=True, alpha=0.4)
    plt.plot(grid, pdf_g, label="Gaussian")
    plt.plot(grid, pdf_s, label="Skew-t (Hansen)")
    plt.title(title)
    plt.xlabel("Rendement")
    plt.ylabel("Densité")
    plt.legend()
    plt.show()

In [ ]:
plot_fit_comparison(train['log_return'], mu_g, sigma_g, skewt_fit, title="Fit comparison - train")

In [ ]:
plot_fit_comparison(test['log_return'], mu_g, sigma_g, skewt_fit, title="Fit comparison - test")

**e. VaR Skew Student sur base d’apprentissage pour alpha = 99%**

In [ ]:
def var_skewt_99_train(skewt_fit, nsim=2_000_000, seed=42):
    rng = np.random.default_rng(seed)
    sim = sample_skewt_hansen(nsim, skewt_fit["mu"], skewt_fit["sigma"], skewt_fit["nu"], skewt_fit["lam"], rng=rng)
    q01 = np.quantile(sim, 0.01)      # 1% quantile (rendements)
    VaR99 = -q01                      # VaR positive (perte)
    return VaR99, q01

VaR99_skewt, q01_skewt = var_skewt_99_train(skewt_fit, nsim=1_000_000, seed=7)
print("Skew-t: q01 (returns) =", q01_skewt, "| VaR99 (loss) =", VaR99_skewt)

# Pour comparaison: VaR gaussienne
q01_gauss = norm.ppf(0.01, loc=mu_g, scale=sigma_g)
VaR99_gauss = -q01_gauss
print("Gaussian: q01 (returns) =", q01_gauss, "| VaR99 (loss) =", VaR99_gauss)

## 4. Protocole de recalibrage

## 5. VaR TVE : Approche Maxima par bloc

**a. Détermination d'une taille de bloc s et construction d'un échantillon de maxima sur la base d’apprentissage**

On doit d'abord convertir les données car en TVE classique, on modélise des maxima. Or en gestion du risque, ce sont les pertes extrêmes (rendements très négatifs) qui nous intéressent. On transforme donc la série de rendements en pertes positives. 

In [ ]:
def convert_returns_to_losses(df, return_col="log_return", loss_col="loss"):
    """
    Convertit les rendements en pertes positives
    pour application de la théorie des valeurs extrêmes.
    """
    df = df.copy()
    df[loss_col] = -df[return_col]
    return df

In [ ]:
df_train_losses = convert_returns_to_losses(train, return_col="log_return")
df_train_losses[[ "log_return", "loss" ]].head()

Nous prendrons une taille de blocs mensuelle, s=21 jours

In [ ]:
def block_maxima(df, loss_col="loss", block_size=21):
    """
    Construit l'échantillon des maxima par blocs (Block Maxima).

    Returns:
    - Series des maxima de blocs
    """
    losses = df[loss_col].values
    n = len(losses)

    # Nombre de blocs complets
    n_blocks = n // block_size

    # On tronque la série pour avoir des blocs complets
    losses_trimmed = losses[: n_blocks * block_size]

    # Reshape (blocs x taille_bloc)
    blocks = losses_trimmed.reshape(n_blocks, block_size)

    # Maxima par bloc
    block_max = blocks.max(axis=1)

    return pd.Series(block_max, name="block_max")

In [ ]:
block_maxima_train = block_maxima(
    df_train_losses,
    loss_col="loss",
    block_size=21
)

print("Nombre de maxima:", len(block_maxima_train))
block_maxima_train.head()

**b. Gumbel plot pour juger de l’hypothèse ξ=0 (i.e. GEV vs EV)**

In [ ]:
def gumbel_plot(block_maxima):
    """
    Gumbel plot pour tester l'hypothèse xi = 0 (EV vs GEV).
    """
    x = np.sort(np.asarray(block_maxima))
    n = len(x)

    # Positions empiriques (plotting positions)
    p = (np.arange(1, n + 1) - 0.5) / n

    # Variable réduite de Gumbel
    y = -np.log(-np.log(p))

    # Plot
    plt.figure(figsize=(6, 6))
    plt.scatter(y, x, s=25)
    
    # Ajustement linéaire (référence visuelle)
    a, b = np.polyfit(y, x, 1)
    #plt.plot(y, a * y + b, linewidth=1)

    plt.xlabel(r"$-\log(-\log(\hat F(x)))$")
    plt.ylabel("Maxima de blocs (pertes)")
    plt.title("Gumbel plot des maxima de blocs")
    plt.grid(alpha=0.3)
    plt.show()

    return a, b


In [ ]:
a_hat, b_hat = gumbel_plot(block_maxima_train)

print("Pente estimée (sigma approx.) :", a_hat)
print("Ordonnée à l'origine (mu approx.) :", b_hat)

Le gumbel plot donne une représentation assez linéaire, l'hypothèse ξ=0 est très plausible. On va donc partir sur une estimation de la loi de Gumbel EV qui est donc pertinente.

**c. Estimation des paramètres de loi EV**

In [ ]:
x = np.asarray(block_maxima_train)
loc_hat, scale_hat = gumbel_r.fit(x)

print("Estimation Gumbel (EV, xi=0)")
print("loc (mu)   =", loc_hat)
print("scale (beta) =", scale_hat)

**d. Validation ex-ante (analyses graphiques, QQ-plot, etc**

Ajustement avec la densité

In [ ]:
grid = np.linspace(x.min(), x.max(), 500)
pdf_fit = gumbel_r.pdf(grid, loc=loc_hat, scale=scale_hat)

plt.figure(figsize=(8,4))
plt.hist(x, bins=15, density=True, alpha=0.4, label="Maxima empiriques")
plt.plot(grid, pdf_fit, label="Gumbel ajustée")
plt.title("Ajustement Gumbel sur les maxima de blocs")
plt.xlabel("Maxima de blocs (pertes)")
plt.ylabel("Densité")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

QQ-plot

In [ ]:
def qqplot_gumbel(block_maxima, loc, scale, title=None):
    """
    QQ-plot des maxima de blocs vs loi de Gumbel ajustée.
    
    block_maxima : array-like, maxima de blocs
    loc, scale   : paramètres MLE de la loi de Gumbel
    """
    x = np.sort(np.asarray(block_maxima))
    n = len(x)

    # Positions empiriques
    p = (np.arange(1, n + 1) - 0.5) / n

    # Quantiles théoriques Gumbel
    q_theo = gumbel_r.ppf(p, loc=loc, scale=scale)

    # QQ-plot
    plt.figure(figsize=(6, 6))
    plt.scatter(q_theo, x, s=25)
    
    # Droite 45°
    mn = min(q_theo.min(), x.min())
    mx = max(q_theo.max(), x.max())
    plt.plot([mn, mx], [mn, mx], linewidth=1)

    plt.xlabel("Quantiles théoriques Gumbel")
    plt.ylabel("Quantiles empiriques (maxima de blocs)")
    plt.title(title if title else "QQ-plot Gumbel")
    plt.grid(alpha=0.3)
    plt.show()


In [ ]:
qqplot_gumbel(
    block_maxima_train,
    loc=loc_hat,
    scale=scale_hat,
    title="QQ-plot des maxima vs Gumbel ajustée"
)

**e. Calcul de la VaR TVE par MB sur base d’apprentissage pour alpha = 99%**

In [ ]:
alpha = 0.99 
s = 21

# Niveau à utiliser pour le quantile des maxima
p_block = alpha ** s

# VaR TVE (sur pertes journalières)
VaR99_loss = gumbel_r.ppf(p_block, loc=loc_hat, scale=scale_hat)

print("p_block =", p_block)
print("VaR 99% (pertes) =", VaR99_loss)

# VaR en rendement (seuil sur r_t)
VaR99_return = -VaR99_loss
print("Seuil VaR 99% en rendements =", VaR99_return)


**f. Protocole permettant de trouver une taille de bloc optimale**

## 6. VaR TVE : Approche Peak over threshold

**a. Fonction permettant d’obtenir le mean excess plot**

In [ ]:
def mean_excess_plot(
    losses,
    min_exceedances=50,
    force_positive_u=True
):
    """
    Mean Excess Plot en testant tous les seuils u = x_(i),
    tout en tronquant à droite via une contrainte N_u >= min_exceedances.
    """
    x = np.asarray(losses)
    x = x[np.isfinite(x)]

    if force_positive_u:
        x = x[x >= 0]

    x_sorted = np.sort(x)
    n = len(x_sorted)

    u_list, e_list, Nu_list = [], [], []

    # On teste chaque valeur comme seuil (sauf les toutes dernières)
    # et on tronque à droite en imposant N_u >= min_exceedances
    for i in range(n - min_exceedances):
        u = x_sorted[i]
        exceedances = x_sorted[i+1:]      # car x_sorted > u (si pas de doublons)
        exceedances = exceedances[exceedances > u]  # robuste aux doublons
        Nu = len(exceedances)

        if Nu < min_exceedances:
            break  # à partir de là, Nu ne fera que diminuer => tronquage à droite

        u_list.append(u)
        e_list.append(np.mean(exceedances - u))
        Nu_list.append(Nu)

    u_list = np.array(u_list)
    e_list = np.array(e_list)
    Nu_list = np.array(Nu_list)

    # Plot + diagnostic du nombre d'excès
    fig, axes = plt.subplots(2, 1, figsize=(9, 7), sharex=True)

    axes[0].plot(u_list, e_list)
    axes[0].set_ylabel("Excès moyen e(u)")
    axes[0].set_title("Mean Excess Plot — tous seuils, tronqué à droite (N_u min)")

    axes[1].plot(u_list, Nu_list)
    axes[1].set_xlabel("Seuil u")
    axes[1].set_ylabel("Nombre d'excès N_u")

    for ax in axes:
        ax.grid(alpha=0.3)

    plt.show()

    return u_list, e_list, Nu_list


**b. Détermination de u par analyse graphique**

In [ ]:
u_vals, e_vals, Nu_vals = mean_excess_plot(
    df_train_losses["loss"],
    min_exceedances=50,
    force_positive_u=True
)

print("Points tracés:", len(u_vals))
print("N_u min/max:", Nu_vals.min(), Nu_vals.max())


In [ ]:
def pick_threshold_by_target_exceedances(u_vals, Nu_vals, target=100):
    idx = np.argmin(np.abs(Nu_vals - target))
    return u_vals[idx], Nu_vals[idx]

u_star, Nu_star = pick_threshold_by_target_exceedances(u_vals, Nu_vals, target=100)
print("Seuil choisi u* =", u_star)
print("Nombre d'excès associé N_u =", Nu_star)


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(u_vals, e_vals, linewidth=1)
plt.axvline(u_star, linestyle="--", linewidth=1, label=f"u* (N_u≈{Nu_star})")
plt.xlabel("Seuil u")
plt.ylabel("Excès moyen e(u)")
plt.title("Mean Excess Plot — seuil sélectionné")
plt.grid(alpha=0.3)
plt.legend()
plt.show()

**c. Estimation des paramètres de loi GPD**

In [ ]:
def build_exceedances(losses, u):
    """
    losses : array-like (pertes X = -r)
    u      : seuil
    Retourne :
      - exc : les excès Y = X - u conditionnellement à X > u
      - Nu  : nombre d'excès
    """
    x = np.asarray(losses)
    x = x[np.isfinite(x)]
    x_exc = x[x > u]
    exc = x_exc - u
    return exc, len(exc)

In [ ]:
# Exemple d'utilisation
exc, Nu = build_exceedances(df_train_losses["loss"], u_star)
print("Seuil u =", u_star)
print("Nombre d'excès Nu =", Nu)
print("Excès (head):", exc[:5])

In [ ]:
from scipy.stats import genpareto

def fit_gpd_mle(exceedances):
    """
    Estime les paramètres GPD (xi, beta) sur des excès >= 0.
    Retourne un dict avec xi (shape) et beta (scale).
    """
    y = np.asarray(exceedances)
    y = y[np.isfinite(y)]
    y = y[y >= 0]

    # Fit MLE: genpareto(c=xi, loc=0, scale=beta)
    xi_hat, loc_hat, beta_hat = genpareto.fit(y, floc=0)

    return {"xi": xi_hat, "beta": beta_hat, "loc": loc_hat}

# Exemple d'utilisation
gpd_fit = fit_gpd_mle(exc)
print("GPD MLE:", gpd_fit)


In [ ]:
y = exc
grid = np.linspace(0, np.max(y), 400)
pdf = genpareto.pdf(grid, c=gpd_fit["xi"], loc=0, scale=gpd_fit["beta"])

plt.figure(figsize=(8,4))
plt.hist(y, bins=30, density=True, alpha=0.4, label="Excès empiriques")
plt.plot(grid, pdf, label="GPD ajustée")
plt.title("Ajustement GPD sur les excès (POT)")
plt.xlabel("Excès y = X - u")
plt.ylabel("Densité")
plt.grid(alpha=0.3)
plt.legend()
plt.show()


In [ ]:
from scipy.stats import genpareto

def qqplot_gpd(exceedances, xi, beta, title=None):
    """
    QQ-plot des excès vs loi GPD ajustée.
    
    exceedances : array-like, Y = X - u (>= 0)
    xi          : paramètre de forme GPD
    beta        : paramètre d'échelle GPD
    """
    y = np.asarray(exceedances)
    y = y[np.isfinite(y)]
    y = y[y >= 0]

    # Quantiles empiriques
    y_sorted = np.sort(y)
    n = len(y_sorted)
    p = (np.arange(1, n + 1) - 0.5) / n

    # Quantiles théoriques GPD
    q_theo = genpareto.ppf(p, c=xi, loc=0, scale=beta)

    # QQ-plot
    plt.figure(figsize=(6, 6))
    plt.scatter(q_theo, y_sorted, s=25)
    
    # Diagonale
    mn = min(q_theo.min(), y_sorted.min())
    mx = max(q_theo.max(), y_sorted.max())
    plt.plot([mn, mx], [mn, mx], linewidth=1)

    plt.xlabel("Quantiles théoriques GPD")
    plt.ylabel("Quantiles empiriques (excès)")
    plt.title(title if title else "QQ-plot GPD (excès POT)")
    plt.grid(alpha=0.3)
    plt.show()

In [ ]:
qqplot_gpd(
    exc,
    xi=gpd_fit["xi"],
    beta=gpd_fit["beta"],
    title="QQ-plot des excès vs GPD ajustée"
)

**e. VaR TVE par PoT sur base d’apprentissage pour alpha = 99%**

In [ ]:
def var_pot_gpd(alpha, u, xi, beta, Nu, n):
    """
    VaR POT basée sur une GPD ajustée sur les excès (X > u).
    alpha : niveau (ex 0.99)
    u     : seuil
    xi    : shape GPD
    beta  : scale GPD
    Nu    : nombre d'excès
    n     : taille totale échantillon
    """
    pu = Nu / n  # fréquence d'excès

    if xi != 0:
        term = ((1 - alpha) / pu) ** (-xi) - 1
        return u + (beta / xi) * term
    else:
        return u + beta * np.log(pu / (1 - alpha))


In [ ]:
# --- Exemple d'utilisation ---
n = len(df_train_losses)  
Nu = len(exc)
xi = gpd_fit["xi"]
beta = gpd_fit["beta"]

VaR99_loss = var_pot_gpd(alpha=0.99, u=u_star, xi=xi, beta=beta, Nu=Nu, n=n)
print("VaR 99% (pertes) =", VaR99_loss)

# Seuil en rendements (optionnel)
VaR99_return = -VaR99_loss
print("Seuil VaR 99% en rendements =", VaR99_return)